In [3]:
import os
import math
import numpy as np
import pandas as pd
# pip install rfit
import rfit
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC, LinearSVC
# from sklearn.tree import DecisionTreeClassifier
#
import warnings
warnings.filterwarnings("ignore")
print("\nReady to continue.")

2024-11-16 15:53:11.193620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Ready to continue.


# 1. Use this Housing Price dataset. 

In [12]:
df = pd.read_csv("/Users/apple/Desktop/DATA37000-24AU/final-project/data/HousePricesAdv/train.csv", header=0)

In [13]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


1. Use this Housing Price dataset. 
- Use SalePrice as target for K-NN regression. 
- For features that are *ORDINAL*, recode them as 0,1,2,... 
- Drop features that are purely categorical.

In [28]:
# Use SalePrice as target for K-NN regression.
# Drop the target and other non-predictive columns
X = df.drop(columns=['SalePrice','Id'])
y = df['SalePrice']

In [29]:
X.shape, y.shape

((1460, 79), (1460,))

In [31]:
# Indentify features as diffferent types

# Categorical features
categorical_features = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
    'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
    'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
    'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
    'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
]

# Ordinal features
ordinal_features = [
    'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence'
]

# Numerical features
numerical_features = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
    'FullBath', 'HalfBath', 'Bedroom', 'Kitchen', 'TotRmsAbvGrd',
    'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
    'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
    'MiscVal', 'MoSold', 'YrSold', 'SalePrice'
]

# Temporal features
temporal_features = ['YearBuilt', 'YearRemodAdd',
                     'GarageYrBlt', 'MoSold', 'YrSold']

# Binary features
binary_features = ['Street', 'Alley', 'CentralAir', 'PavedDrive']

# Target feature
target_feature = ['SalePrice']

In [32]:
# For features that are *ORDINAL*, recode them as 0,1,2,...
# Ordinal mappings
ordinal_mappings = {
    "OverallQual": {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9},
    "OverallCond": {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9},
    "ExterQual": {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "ExterCond": {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "BsmtQual": {"NA": -1, "Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "BsmtCond": {"NA": -1, "Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "BsmtExposure": {"NA": -1, "No": 0, "Mn": 1, "Av": 2, "Gd": 3},
    "BsmtFinType1": {"NA": -1, "Unf": 0, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 4, "GLQ": 5},
    "BsmtFinType2": {"NA": -1, "Unf": 0, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 4, "GLQ": 5},
    "HeatingQC": {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "KitchenQual": {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "FireplaceQu": {"NA": -1, "Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "GarageQual": {"NA": -1, "Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "GarageCond": {"NA": -1, "Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "PoolQC": {"NA": -1, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "Fence": {"NA": -1, "MnWw": 0, "GdWo": 1, "MnPrv": 2, "GdPrv": 3}
}

# Recode ordinal features
for feature, mapping in ordinal_mappings.items():
    if feature in X.columns:
        X[feature] = X[feature].replace(mapping)

X[ordinal_features]

,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC,Fence
0,6,4,3,2,3.0,2.0,0.0,5.0,0.0,4,3,NaN,2.0,2.0,NaN,NaN
1,5,7,2,2,3.0,2.0,3.0,4.0,0.0,4,2,2.0,2.0,2.0,NaN,NaN
2,6,4,3,2,3.0,2.0,1.0,5.0,0.0,4,3,2.0,2.0,2.0,NaN,NaN
3,6,4,2,2,2.0,3.0,0.0,4.0,0.0,3,3,3.0,2.0,2.0,NaN,NaN
4,7,4,3,2,3.0,2.0,2.0,5.0,0.0,4,3,2.0,2.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5,4,2,2,3.0,2.0,0.0,0.0,0.0,4,2,2.0,2.0,2.0,NaN,NaN
1456,5,5,2,2,3.0,2.0,0.0,4.0,2.0,2,2,2.0,2.0,2.0,NaN,2.0
1457,6,8,4,3,2.0,3.0,0.0,5.0,0.0,4,3,3.0,2.0,2.0,NaN,3.0
1458,4,5,2,2,2.0,2.0,1.0,5.0,2.0,3,3,NaN,2.0,2.0,NaN,NaN


In [33]:
# Drop features that are purely categorical
X_original = X.copy()
X = X_original.drop(columns=categorical_features)

# Check the shapes of X_original and X
print(f"Original X shape (X_original): {X_original.shape}")
print(f"Modified X shape (X): {X.shape}")

Original X shape (X_original): (1460, 79)
Modified X shape (X): (1460, 48)
